In [2]:
import torch

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Parameter
from functools import wraps

from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from torch.optim import lr_scheduler

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math

import copy

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

import copy
import numpy
import pandas

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### data set definitions

In [6]:
df_each_household = pd.read_csv('../../4차년도 가전기기 데이터/101_302_.csv', index_col='index', infer_datetime_format=True)
df_each_household.index = pd.to_datetime(df_each_household.index)

In [8]:
df_each_household.columns

Index(['PMM(조명PMM)', 'TV(거실)', '노트북(침실2)', '세탁기(발코니2)', '세탁기_건조기(주방)',
       '셋톱박스(거실)', '에어컨(거실)', '에어프라이기(주방)', '이동형에어컨(침실3)', 'presence_USM(거실)',
       'presence_USM(방1)', 'presence_USM(방3)', 'presence_USM(방4)',
       'humidity_USM(거실)', 'humidity_USM(방1)', 'humidity_USM(방3)',
       'humidity_USM(방4)', 'temperature_USM(거실)', 'temperature_USM(방1)',
       'temperature_USM(방3)', 'temperature_USM(방4)', 'power_this_month',
       'month', 'day_of_the_week', 'holiday', 'hour'],
      dtype='object')

In [10]:
# 101-302
appliance_columns = ['PMM(조명PMM)', 'TV(거실)', '노트북(침실2)', '세탁기(발코니2)', '세탁기_건조기(주방)',
                     '셋톱박스(거실)']

# # 101-402
# appliance_columns = ['PC(침실2)', 'PMM(조명)', 'TV(거실)', 'TV(침실1)', '선풍기1(침실1)', '선풍기2(침실2)',
#                      '세탁기(발코니2)', '에어컨(거실)', '에어컨실외기(거실)', '에어프라이기(주방)', '전기밥솥(주방)',
#                      '전자레인지(주방)', '커피포트(주방)', '헤어드라이어(침실1)']

# 101-502
# appliance_columns = ['PMM(조명PMM)', 'TV(거실)', 'TV(방1)', 'TV(방2)', '공기청정기(거실)', '밥솥(주방)',
#                      '세탁기(발코니2)', '에어컨(거실)', '오븐(주방)', '인덕션(주방)', '커피포트(주방)']

# # 101-702
# appliance_columns = ['PC(방2)', 'PC(방3)', 'PC(방4)', 'PMM(주방)', 'TV(거실)', '선풍기(방2)',
#                      '세탁기(발코니2)', '에어컨(거실)', '에어프라이어(주방)', '전기오븐(주방)', '전자레인지(주방)']

# # 101-1802
# appliance_columns = ['TV(거실)', 'TV(안방)', '공기청정기(거실)', '안마기(거실)', '에어컨 실외기(거실)', '에어컨(거실)',
#                      '에어컨(침실2)', '에어프라이어(주방)', '헤어드라이어(안방)']

# # 102-202
# appliance_columns = ['PC(침실3)', 'PMM(식당)', 'TV(거실)', '세탁기(방2)', '식기세척기(주방)', '전기밥솥1(주방)',
#                      '전기장판(침실1)', '커피포트(방3)']

# # 102-207
# appliance_columns = ['TV(거실)', 'TV(안방)', '에어컨(거실)', '에어컨(안방)', '전기밥솥(주방)']

# # 102-208
# appliance_columns = ['PMM(주방)', 'TV(거실)', '노트북(방1)', '선풍기(거실)', '선풍기(방1)', '선풍기(방3)',
#                      '에어컨(거실)', '전기밥솥(주방)', '전자레인지(주방)']

# # 102-303
# appliance_columns = ['PC(거실)', 'PMM(주방)', 'TV(침실1)', '거실에어컨(거실)', '공기청정기(거실)', '세탁기(발코니2)',
#                      '에어프라이어(주방)', '전기밥솥(주방)', '전자레인지(주방)']

# # 102-506
# appliance_columns = ['PMM(조명)', 'TV(거실)', 'TV(침실1)', '세탁기(발코니2)', '에어컨(거실)', '전기밥솥(주방)',
#                      '커피포트(주방)', '컴퓨터(방3)', '컴퓨터(침실3)']

# # 102-609
# appliance_columns = ['PC(방1)', 'PMM(주방)', 'TV(거실)', '공기청정기(거실)', '세탁기(발코니2)']

# # 102-707
# appliance_columns = ['PC1(침실2)', 'PC2(침실2)', 'PMM(주방)', 'TV(거실)', 'TV1(침실1)', '밥솥(주방)','세탁기(발코니)']

# # 102-709
# appliance_columns = ['PMM(주방)', 'TV(거실)', '공기청정기(거실)', '밥솥(주방)', '세탁기(발코니2)', '에어컨(거실)']

# # 102-1101
# appliance_columns = ['PMM(주방)', 'TV(거실)', '공기청정기(방1)', '밥솥(주방)', '세탁기(발코니2)', '에어컨(거실)', '커피포터(주방)']

# # 102-1403
# appliance_columns = ['PMM(주방)', 'TV(거실)', '선풍기(거실)', '세탁기(발코니2)', '셋톱박스(거실)', '에어컨(거실)', '전기오븐(주방)', '전자레인지(주방)']

# # 102-1409
# appliance_columns = ['PMM(주방)', 'TV(거실)', '에어컨(거실)']

# # 102-1504
# appliance_columns = ['PC(일체형)(침실3)', 'PMM(주방)', 'TV1(거실)', 'TV2(침실1)', '건조기(발코니2)',
#                      '서큘레이터(침실1)', '세탁기(발코니2)', '스탠드 에어컨(거실)', '전기밥솥(주방)', '전자레인지(주방)']

# # 102-1703
# appliance_columns = ['TV(안방)', '공기청정기(거실)', '공기청정기(침실2)', '선풍기(침실2)', '선풍기(침실3)',
#                      '에어컨 실외기(거실)', '에어컨(거실)', '에어컨(안방)', '전자레인지(주방)']

# # 102-1808
# appliance_columns = ['TV(거실)', 'TV(침실1)', '세탁기(발코니2)', '전기밥솥(주방)', '헤어드라이어(침실1)']

time_columns = ['month', 'day_of_the_week', 'holiday', 'hour']



df_each_household_appliance = df_each_household[appliance_columns]

In [11]:
df_each_household_appliance

,PMM(조명PMM),TV(거실),노트북(침실2),세탁기(발코니2),세탁기_건조기(주방),셋톱박스(거실)
index,,,,,,
2021-12-01 00:00:00,472.0,55.9,10.8,1.4,0.6,4.6
2021-12-01 00:05:00,472.0,58.0,12.2,1.5,0.5,4.5
2021-12-01 00:10:00,473.0,57.5,9.6,0.9,0.7,4.5
2021-12-01 00:15:00,473.0,58.3,10.3,1.3,0.8,4.7
2021-12-01 00:20:00,472.0,56.9,9.8,1.0,0.7,4.6
...,...,...,...,...,...,...
2022-11-30 23:35:00,NaN,57.2,NaN,NaN,NaN,4.8
2022-11-30 23:40:00,NaN,53.8,NaN,NaN,NaN,4.1
2022-11-30 23:45:00,NaN,1.0,NaN,NaN,NaN,3.9


In [15]:
# 시간 변수
time_columns = ['month', 'day_of_the_week', 'holiday', 'hour']

# 타겟 변수
#target_columns = ['presence_USM(침실1)']
target_columns = ['presence_USM(거실)']
#target_columns = ['presence_USM(거실인체감지)']
#target_columns = ['presence_USM(안방)']

df_each_household_appliance = df_each_household[appliance_columns]
df_each_household_appliance = df_each_household_appliance.fillna(0)

df_temp = pd.DataFrame()

# (t-6), (t-5), ... (t-1)에 대한 가전기기 사용 패턴 데이터 파생변수 추가
steps = 6
for step in np.arange(steps):
    appliance_time_columns = [f"{i}_(t-{step+1})" for i in appliance_columns]
    df_each_household_appliance_temp = df_each_household_appliance.copy()
    df_each_household_appliance_temp = df_each_household_appliance_temp.shift(+(step+1))
    df_each_household_appliance_temp.columns = appliance_time_columns
    df_temp = pd.concat([df_temp,df_each_household_appliance_temp], axis=1)

# 시간 변수 및 타겟 변수 추가
df_temp = pd.merge(df_temp, df_each_household[time_columns], left_index=True, right_index=True, how='left')
df_temp = pd.merge(df_temp, df_each_household[target_columns], left_index=True, right_index=True, how='left') 

# 결측치 제거

#df_temp = df_temp.fillna(0)
df_temp = df_temp.dropna()

# train : '2021-12-01':'2022-10-31'
# test : '2022-11-01':'2022-12-31'
df_household_train = df_temp['2021-12-01':'2022-10-31'] 
df_household_test = df_temp['2022-11-01':'2022-12-31'] 

print(f"train 갯수 : {len(df_household_train)}")
print(f"test 갯수 : {len(df_household_test)}")

train 갯수 : 89643
test 갯수 : 6443


In [17]:
df_household_train

,PMM(조명PMM)_(t-1),TV(거실)_(t-1),노트북(침실2)_(t-1),세탁기(발코니2)_(t-1),세탁기_건조기(주방)_(t-1),셋톱박스(거실)_(t-1),PMM(조명PMM)_(t-2),TV(거실)_(t-2),노트북(침실2)_(t-2),세탁기(발코니2)_(t-2),...,TV(거실)_(t-6),노트북(침실2)_(t-6),세탁기(발코니2)_(t-6),세탁기_건조기(주방)_(t-6),셋톱박스(거실)_(t-6),month,day_of_the_week,holiday,hour,presence_USM(거실)
index,,,,,,,,,,,,,,,,,,,,,
2021-12-01 00:30:00,472.0,58.4,10.1,1.2,0.6,4.6,472.0,56.9,9.8,1.0,...,55.9,10.8,1.4,0.6,4.6,12,2,0,0,0.0
2021-12-01 00:35:00,468.0,57.9,12.6,1.1,0.8,4.9,472.0,58.4,10.1,1.2,...,58.0,12.2,1.5,0.5,4.5,12,2,0,0,1.0
2021-12-01 00:40:00,548.0,58.3,0.0,0.9,0.7,4.6,468.0,57.9,12.6,1.1,...,57.5,9.6,0.9,0.7,4.5,12,2,0,0,1.0
2021-12-01 00:45:00,549.0,57.8,0.0,1.1,0.6,4.6,548.0,58.3,0.0,0.9,...,58.3,10.3,1.3,0.8,4.7,12,2,0,0,0.0
2021-12-01 00:50:00,16.0,57.6,0.0,0.9,0.7,4.6,549.0,57.8,0.0,1.1,...,56.9,9.8,1.0,0.7,4.6,12,2,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 23:35:00,913.0,57.4,0.0,0.0,0.0,4.7,1079.0,57.0,0.0,0.0,...,56.9,0.0,0.0,0.0,4.6,10,0,0,23,1.0
2022-10-31 23:40:00,914.0,54.7,0.0,0.0,0.0,4.7,913.0,57.4,0.0,0.0,...,57.0,0.0,0.0,0.0,4.6,10,0,0,23,0.0
2022-10-31 23:45:00,911.0,57.1,0.0,0.0,0.0,4.8,914.0,54.7,0.0,0.0,...,57.1,0.0,0.0,0.0,4.6,10,0,0,23,0.0


## uncertainty

### Intro

- What is Bayesian deep learning?
- What is uncertainty?
- Why is uncertainty important?

### What is uncertainty?

- Epistemic Uncertainty in Bayesian Deep Learning

- Heteroscedastic Aleatoric Uncertainty



### Why is uncertainty important?

### Calculating uncertainty in deep learning classification models

In [19]:
# standard categorical cross entropy
# N data points, C classes
# true - true values. Shape: (N, C)
# pred - predicted values. Shape: (N, C)
# returns - loss (N)

def categorical_cross_entropy(true, pred):
    return np.sum(true * np.log(pred), axis=1)

In [22]:
# Bayesian categorical cross entropy.
# N data points, C classes, T monte carlo simulations
# true - true values. Shape: (N, C)
# pred_var - predicted logit values and variance. Shape: (N, C + 1)
# returns - loss (N,)

def bayesian_categorical_crossentropy(T, num_classes):
    def bayesian_categorical_crossentropy_internal(true, pred_var):
        # shape: (N,)
        std = torch.sqrt(pred_var[:, num_classes:])
        # shape: (N,)
        variance = pred_var[:, num_classes]
        variance_depressor = torch.exp(variance) - torch.ones_like(variance)
        # shape: (N, C)
        pred = pred_var[:, 0:num_classes]
        # shape: (N,)
        undistorted_loss = K.categorical_crossentropy(pred, true, from_logits=True)
        # shape: (T,)
        iterable = K.variable(np.ones(T))
        dist = distributions.Normal(loc=K.zeros_like(std), scale=std)
        monte_carlo_results = K.map_fn(gaussian_categorical_crossentropy(true, pred, dist, undistorted_loss, num_classes), iterable, name='monte_carlo_results')

        variance_loss = K.mean(monte_carlo_results, axis=0) * undistorted_loss
    
        return variance_loss + undistorted_loss + variance_depressor
    return bayesian_categorical_crossentropy_internal

In [34]:
#pred_var[:, num_classes:] = 

torch.ones_like(torch.tensor([4,4]))

tensor([1, 1])

In [51]:
num_classes = C = 2
N = 3

true= torch.tensor([[0.3, 0.1], [0.3, 0.1], [0.3, 0.1]])
pred_var = torch.tensor([[0.3, 0.1], [0.3, 0.1], [0.3, 0.1]])

print(f"true's size : {true.size()}, pred_var's size : {pred_var.size()}")

true's size : torch.Size([3, 2]), pred_var's size : torch.Size([3, 2])
